# Compose: Training a model to generate music

In [1]:
import os
import pickle
import numpy
from music21 import note, chord

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model

from models.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network

2022-10-21 15:36:37.537279: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Set parameters

In [6]:
data_folder = "/app/data/bach-cello"
store_folder = "/app/notebooks/music/bach-cello/store"
mode = "build"  # 'load' #



# model params
embed_size = 100
rnn_units = 256
use_attention = True

## Extract the notes

In [7]:
if mode == "build":

    music_list, parser = get_music_list(data_folder)
    print(len(music_list), "files in total")

    notes = []
    durations = []

    for i, file in enumerate(music_list):
        print(i + 1, "Parsing %s" % file)
        original_score = parser.parse(file).chordify()

        for interval in intervals:

            score = original_score.transpose(interval)

            notes.extend(["START"] * seq_len)
            durations.extend([0] * seq_len)

            for element in score.flat:

                if isinstance(element, note.Note):
                    if element.isRest:
                        notes.append(str(element.name))
                        durations.append(element.duration.quarterLength)
                    else:
                        notes.append(str(element.nameWithOctave))
                        durations.append(element.duration.quarterLength)

                if isinstance(element, chord.Chord):
                    notes.append(".".join(n.nameWithOctave for n in element.pitches))
                    durations.append(element.duration.quarterLength)

    with open(os.path.join(data_folder, "notes"), "wb") as f:
        pickle.dump(notes, f)  # ['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(data_folder, "durations"), "wb") as f:
        pickle.dump(durations, f)
else:
    with open(os.path.join(store_folder, "notes"), "rb") as f:
        notes = pickle.load(f)  # ['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, "durations"), "rb") as f:
        durations = pickle.load(f)

36 files in total
1 Parsing /app/data/bach-cello/cs1-1pre.mid
2 Parsing /app/data/bach-cello/cs1-2all.mid
3 Parsing /app/data/bach-cello/cs1-3cou.mid
4 Parsing /app/data/bach-cello/cs1-4sar.mid
5 Parsing /app/data/bach-cello/cs1-5men.mid
6 Parsing /app/data/bach-cello/cs1-6gig.mid
7 Parsing /app/data/bach-cello/cs2-1pre.mid
8 Parsing /app/data/bach-cello/cs2-2all.mid
9 Parsing /app/data/bach-cello/cs2-3cou.mid
10 Parsing /app/data/bach-cello/cs2-4sar.mid
11 Parsing /app/data/bach-cello/cs2-5men.mid
12 Parsing /app/data/bach-cello/cs2-6gig.mid
13 Parsing /app/data/bach-cello/cs3-1pre.mid
14 Parsing /app/data/bach-cello/cs3-2all.mid
15 Parsing /app/data/bach-cello/cs3-3cou.mid
16 Parsing /app/data/bach-cello/cs3-4sar.mid
17 Parsing /app/data/bach-cello/cs3-5bou.mid
18 Parsing /app/data/bach-cello/cs3-6gig.mid
19 Parsing /app/data/bach-cello/cs4-1pre.mid
20 Parsing /app/data/bach-cello/cs4-2all.mid
21 Parsing /app/data/bach-cello/cs4-3cou.mid
22 Parsing /app/data/bach-cello/cs4-4sar.mid
2

## Create the lookup tables

In [ ]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, "distincts"), "wb") as f:
    pickle.dump(distincts, f)

# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, "lookups"), "wb") as f:
    pickle.dump(lookups, f)

In [12]:
print("\nnote_to_int")
note_to_int


note_to_int


{'A2': 0,
 'A2.A3': 1,
 'A2.B-3': 2,
 'A2.B2': 3,
 'A2.C3': 4,
 'A2.D3': 5,
 'A2.E-3': 6,
 'A2.E3': 7,
 'A2.E3.A3': 8,
 'A2.E3.C#4': 9,
 'A2.E3.C#4.A4': 10,
 'A2.E3.C#4.E4': 11,
 'A2.E3.C#4.G#4': 12,
 'A2.E3.C4': 13,
 'A2.E3.D4': 14,
 'A2.F#3': 15,
 'A2.F#3.C4': 16,
 'A2.F#3.D4': 17,
 'A2.F#3.D4.A4': 18,
 'A2.F#3.D4.E4': 19,
 'A2.F#3.D4.F#4': 20,
 'A2.F#4': 21,
 'A2.F3': 22,
 'A2.F3.C4': 23,
 'A2.F3.D4': 24,
 'A2.F3.D4.A4': 25,
 'A2.G3': 26,
 'A2.G3.C#4': 27,
 'A2.G3.D4': 28,
 'A3': 29,
 'A3.B-3': 30,
 'A3.B3': 31,
 'A3.B3.C#4': 32,
 'A3.B3.C#4.D4': 33,
 'A3.B3.C4': 34,
 'A3.C#4': 35,
 'A3.C#4.E4': 36,
 'A3.C4': 37,
 'A3.D4': 38,
 'A3.E4': 39,
 'A3.E4.F#4': 40,
 'A3.E4.G4': 41,
 'A3.F#4': 42,
 'A3.F#4.G4': 43,
 'A3.F4': 44,
 'A3.G4': 45,
 'A4': 46,
 'A4.B4': 47,
 'B-2': 48,
 'B-2.A3': 49,
 'B-2.B-3': 50,
 'B-2.C3': 51,
 'B-2.D3': 52,
 'B-2.D3.A3': 53,
 'B-2.D3.E-3.G#3': 54,
 'B-2.D3.G#3': 55,
 'B-2.E-3': 56,
 'B-2.E3': 57,
 'B-2.E3.D4': 58,
 'B-2.F#3.C#4.E4': 59,
 'B-2.F3': 60,
 'B-2.F

In [13]:
print("\nduration_to_int")
duration_to_int


duration_to_int


{0: 0,
 Fraction(1, 12): 1,
 Fraction(1, 6): 2,
 0.25: 3,
 Fraction(1, 3): 4,
 0.5: 5,
 Fraction(2, 3): 6,
 0.75: 7,
 1.0: 8,
 1.25: 9,
 Fraction(4, 3): 10,
 1.5: 11,
 1.75: 12,
 2.0: 13,
 2.5: 14,
 3.0: 15,
 4.0: 16}

## Prepare the sequences used by the Neural Network

In [14]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [15]:
print("pitch input")
print(network_input[0][0])
print("duration input")
print(network_input[1][0])
print("pitch output")
print(network_output[0][0])
print("duration output")
print(network_output[1][0])

pitch input
[469 469 469 469 469 469 469 469 469 469 469 469 469 469 469 469 469 469
 469 469 469 469 469 469 469 469 469 469 469 469 469 469]
duration input
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
pitch output
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## Create the structure of the neural network

In [16]:
model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)
model.summary()

2022-10-21 16:10:30.869779: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    47000       ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    1700        ['input_2[0][0]']                
                                                                                              

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [17]:
# Currently errors in TF2.2
# plot_model(model, to_file=os.path.join(run_folder ,'viz/model.png'), show_shapes = True, show_layer_names = True)

## Train the neural network

In [18]:
weights_folder = os.path.join(run_folder, "weights")
# model.load_weights(os.path.join(weights_folder, "weights.h5"))

In [19]:
weights_folder = os.path.join(run_folder, "weights")

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor="loss",
    verbose=0,
    save_best_only=True,
    mode="min",
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"), monitor="loss", verbose=0, save_best_only=True, mode="min"
)

early_stopping = EarlyStopping(monitor="loss", restore_best_weights=True, patience=10)


callbacks_list = [checkpoint1, checkpoint2, early_stopping]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(
    network_input,
    network_output,
    epochs=2000000,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks_list,
    shuffle=True,
)

FileNotFoundError: [Errno 2] Unable to create file (unable to open file: name = 'run/compose/0006_cello/weights/weights.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)